In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46037
2,Huelva,Confirmados PDIA 14 días,312
3,Huelva,Tasa PDIA 14 días,"60,79856577742269"
4,Huelva,Confirmados PDIA 7 días,161
5,Huelva,Tasa PDIA 7 dias,"31,373618878734142"
6,Huelva,Total Confirmados,46240
7,Huelva,Curados,45045
8,Huelva,Fallecidos,419


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46037.0


/tmp/ipykernel_4655/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12653.0


/tmp/ipykernel_4655/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4655/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4655/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4655/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4110 personas en los últimos 7 días 

Un positivo PCR cada 2358 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46037.0,161.0,312.0,513170.0,31.373619,60.798566,29.0
Huelva-Costa,27231.0,95.0,182.0,289548.0,32.809759,62.856590,16.0
Condado-Campiña,14300.0,43.0,84.0,156231.0,27.523347,53.766538,11.0
Cartaya,2101.0,24.0,61.0,20083.0,119.504058,303.739481,6.0
Huelva (capital),12653.0,35.0,61.0,143837.0,24.333099,42.409116,4.0
Palos de la Frontera,1065.0,12.0,17.0,11742.0,102.197241,144.779424,4.0
Isla Cristina,3156.0,10.0,17.0,21393.0,46.744262,79.465246,3.0
Palma del Condado (La),1434.0,4.0,11.0,10801.0,37.033608,101.842422,3.0
Ayamonte,2105.0,10.0,15.0,21104.0,47.384382,71.076573,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,45.0,14.0,23.0,713.0,1963.534362,3225.806452,0.0
Granado (El),45.0,5.0,5.0,532.0,939.849624,939.849624,1.0
Cartaya,2101.0,24.0,61.0,20083.0,119.504058,303.739481,6.0
Zalamea la Real,123.0,1.0,8.0,3054.0,32.743942,261.951539,0.0
Minas de Riotinto,184.0,4.0,6.0,3812.0,104.931794,157.397692,2.0
Palos de la Frontera,1065.0,12.0,17.0,11742.0,102.197241,144.779424,4.0
Moguer,2009.0,8.0,24.0,21867.0,36.584808,109.754424,1.0
Palma del Condado (La),1434.0,4.0,11.0,10801.0,37.033608,101.842422,3.0
Isla Cristina,3156.0,10.0,17.0,21393.0,46.744262,79.465246,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Zalamea la Real,123.0,1.0,8.0,3054.0,32.743942,261.951539,0.0,0.125000
Punta Umbría,1277.0,1.0,4.0,15355.0,6.512537,26.050147,0.0,0.250000
Moguer,2009.0,8.0,24.0,21867.0,36.584808,109.754424,1.0,0.333333
Palma del Condado (La),1434.0,4.0,11.0,10801.0,37.033608,101.842422,3.0,0.363636
Cartaya,2101.0,24.0,61.0,20083.0,119.504058,303.739481,6.0,0.393443
Bollullos Par del Condado,1297.0,2.0,5.0,14387.0,13.901439,34.753597,1.0,0.400000
Aracena,673.0,1.0,2.0,8255.0,12.113870,24.227741,0.0,0.500000
Sierra de Huelva-Andévalo Central,4080.0,23.0,46.0,67391.0,34.129186,68.258373,2.0,0.500000
Cortegana,293.0,1.0,2.0,4602.0,21.729683,43.459365,0.0,0.500000
